In [1]:
import os,sys
sys.path.append("../../")
from analysis_common import *
from root_common import *
from analysis_helpers import *
%jsroot on

#Definitions
lep_accept_pt = 0.4 #GeV
lep_accept_angle = radians(40)
hadron_accept_ke = 0.060 #GeV
hadron_accept_angle = radians(40)

dir_files_G18_02a = "/Data/ldmx_eTi_4GeV_G18_02a_00"
dir_files_GEM21_11b = "/Data/ldmx_eTi_4GeV_GEM21_11b_00"
gst_files_G18_02a = glob.glob(f"{dir_files_G18_02a}/*gst.root")
gst_files_GEM21_11b = glob.glob(f"{dir_files_GEM21_11b}/*gst.root")

gst_chain_GEM21_11b = create_gst_chain(gst_files_GEM21_11b,verbose=True)
gst_chain_G18_02a = create_gst_chain(gst_files_G18_02a,verbose=True)

events = 100000
df_gst_GEM21_11b_all = ROOT.RDataFrame(gst_chain_GEM21_11b)
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Range(events)

df_gst_G18_02a_all = ROOT.RDataFrame(gst_chain_G18_02a)
df_gst_G18_02a_all = df_gst_G18_02a_all.Range(events)

Welcome to JupyROOT 6.24/06
Created gst chain from 10 files with 1000000 total events.
Created gst chain from 10 files with 1000000 total events.


In [2]:
#define a function that makes new lepton variables
def define_df_gst_lep_vars(df_gst):
    df_gst = df_gst.Define("ptl","sqrt(pxl*pxl+pyl*pyl)") #pt lepton
    df_gst = df_gst.Define("thetazl","atan2(ptl,pzl)") #theta_z of lepton
    df_gst = df_gst.Define("energy_transfer","Ev-El")
    return df_gst
#apply definitions for new lepton variables
df_gst_GEM21_11b_all = define_df_gst_lep_vars(df_gst_GEM21_11b_all)
df_gst_G18_02a_all = define_df_gst_lep_vars(df_gst_G18_02a_all)

#define a function to make a bunch of hadron related variables
#sfx is the suffix on the vars (initial, final)
def define_df_gst_hadron_vars(df_gst,sfx=["i","f"]):
    
    for s in sfx:
        df_gst = df_gst.Define(f"thetaxz{s}",f"atan2(px{s},pz{s})")
        df_gst = df_gst.Define(f"thetayz{s}",f"atan2(py{s},pz{s})")
        df_gst = df_gst.Define(f"pt{s}",f"sqrt(px{s}*px{s}+py{s}*py{s})")
        
        #note, only for 'i' is total momentum not already in the gst tree
        if s=="i":
            df_gst = df_gst.Define(f"p{s}",f"sqrt(px{s}*px{s}+py{s}+py{s}+pz{s}*pz{s})")
            
        df_gst = df_gst.Define(f"thetaz{s}",f"atan2(pt{s},pz{s})")
        df_gst = df_gst.Define(f"mass{s}",f"sqrt(E{s}*E{s}-p{s}*p{s})")
        df_gst = df_gst.Define(f"ke{s}",f"E{s}-mass{s}")
        
        #for the hadrons, get the indices sorted by KE
        df_gst = df_gst.Define(f"idx_ke{s}",f"Reverse(Argsort(ke{s}))")
                
    return df_gst
df_gst_GEM21_11b_all = define_df_gst_hadron_vars(df_gst_GEM21_11b_all)
df_gst_G18_02a_all = define_df_gst_hadron_vars(df_gst_G18_02a_all)

#Giana's added definitions
#num of pions (+ & - & neutral)
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("num_pions", "nfpip+nfpim+nfpi0") 
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("num_pions", "nfpip+nfpim+nfpi0") 
#ke of proton and num of protons
df_gst_GEM21_11b_all.Define("kef_proton","kef[pdgf==2212]").Define("n_proton","kef_proton.size()") #is this diff from nfp?
df_gst_G18_02a_all.Define("kef_proton","kef[pdgf==2212]").Define("n_proton","kef_proton.size()") 
#ke of neutrons and num of neutrons
df_gst_GEM21_11b_all.Define("kef_neutron", "kef[pdgf==2112]").Define("n_neutron", "kef_neutron.size()")
df_gst_G18_02a_all.Define("kef_neutron", "kef[pdgf==2112]").Define("n_neutron", "kef_neutron.size()")
#most energetic proton
df_gst_GEM21_11b_all.Define("kef_proton_max", "Max(kef[pdgf==2212])")
df_gst_G18_02a_all.Define("kef_proton_max", "Max(kef[pdgf==2212])")
#most energetic neutron
df_gst_GEM21_11b_all.Define("kef_neutron_max", "Max(kef[pdgf==2112])")
df_gst_G18_02a_all.Define("kef_neutron_max", "Max(kef[pdgf==2112])")
#kef of protons
df_gst_GEM21_11b_all.Define("kef_above_threshold_protons",f"kef[kef>{hadron_accept_ke}&&pdgf==2212]")
df_gst_G18_02a_all.Define("kef_above_threshold_protons",f"kef[kef>{hadron_accept_ke}&&pdgf==2212]")

<cppyy.gbl.ROOT.RDF.RInterface<ROOT::Detail::RDF::RRange<ROOT::Detail::RDF::RLoopManager>,void> object at 0x68af140>

In [3]:
# plot of lep pt momenta for interaction types for g18 and gem models
b = ROOT.TCanvas("b")
x_start = 0
x_end = 1.6
bins = 100
df_gst_G18_02a_leppt = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt}")
G18_lep_pt_QEL = df_gst_G18_02a_leppt.Filter("qel==1").Histo1D(("G18_lep_pt_QEL", "Transverse Momenta in QE Interactions in G18 Model", bins, x_start, x_end), "ptl")
G18_lep_pt_QEL.SetLineColor(ROOT.kViolet)
G18_lep_pt_QEL.Draw("same")

G18_lep_pt_MEC = df_gst_G18_02a_leppt.Filter("mec==1").Histo1D(("G18_lep_pt_MEC", "", bins, x_start, x_end), "ptl")
G18_lep_pt_MEC.SetLineColor(ROOT.kRed)
G18_lep_pt_MEC.Draw("same")

G18_lep_pt_RES = df_gst_G18_02a_leppt.Filter("res==1").Histo1D(("G18_lep_pt_RES", "", bins, x_start, x_end), "ptl")
G18_lep_pt_RES.SetLineColor(ROOT.kGreen)
G18_lep_pt_RES.Draw("same")

G18_lep_pt_DIS = df_gst_G18_02a_leppt.Filter("dis==1").Histo1D(("G18_lep_pt_DIS", "", bins, x_start, x_end), "ptl")
G18_lep_pt_DIS.SetLineColor(ROOT.kBlue)
G18_lep_pt_DIS.Draw("same")

G18_lep_pt_none = df_gst_G18_02a_leppt.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_lep_pt_none", "", bins, x_start, x_end), "ptl")
G18_lep_pt_none.SetLineColor(ROOT.kGray)
G18_lep_pt_none.Draw("same")

G18_lep_pt_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
G18_lep_pt_leg.AddEntry("G18_lep_pt_QEL", "QE", "l")
G18_lep_pt_leg.AddEntry("G18_lep_pt_MEC", "MEC", "l")
G18_lep_pt_leg.AddEntry("G18_lep_pt_RES", "RES", "l")
G18_lep_pt_leg.AddEntry("G18_lep_pt_DIS", "DIS", "l")
G18_lep_pt_leg.AddEntry("G18_lep_pt_none", "Uncatagorized", "l")
G18_lep_pt_leg.Draw()

b.SetLogy()
b.Draw()

e = ROOT.TCanvas("e")
df_gst_GEM21_11b_leppt = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt}")
GEM_lep_pt_QEL = df_gst_GEM21_11b_leppt.Filter("qel==1").Histo1D(("GEM_lep_pt_QEL", "Transverse Momenta in QE Interactions in GEM21 Model", bins, x_start, x_end), "ptl")
GEM_lep_pt_QEL.SetLineColor(ROOT.kViolet)
GEM_lep_pt_QEL.Draw("same")

GEM_lep_pt_MEC = df_gst_GEM21_11b_leppt.Filter("mec==1").Histo1D(("GEM_lep_pt_MEC", "", bins, x_start, x_end), "ptl")
GEM_lep_pt_MEC.SetLineColor(ROOT.kRed)
GEM_lep_pt_MEC.Draw("same")

GEM_lep_pt_RES = df_gst_GEM21_11b_leppt.Filter("res==1").Histo1D(("GEM_lep_pt_RES", "", bins, x_start, x_end), "ptl")
GEM_lep_pt_RES.SetLineColor(ROOT.kGreen)
GEM_lep_pt_RES.Draw("same")

GEM_lep_pt_DIS = df_gst_GEM21_11b_leppt.Filter("dis==1").Histo1D(("GEM_lep_pt_DIS", "", bins, x_start, x_end), "ptl")
GEM_lep_pt_DIS.SetLineColor(ROOT.kBlue)
GEM_lep_pt_DIS.Draw("same")

GEM_lep_pt_none = df_gst_GEM21_11b_leppt.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("GEM_lep_pt_none", "", bins, x_start, x_end), "ptl")
GEM_lep_pt_none.SetLineColor(ROOT.kGray)
GEM_lep_pt_none.Draw("same")

GEM_lep_pt_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_QEL", "QE", "l")
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_MEC", "MEC", "l")
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_RES", "RES", "l")
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_DIS", "DIS", "l")
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_none", "Uncatagorized", "l")
GEM_lep_pt_leg.Draw()

e.SetLogy()
e.Draw()

Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used


In [4]:
#num of pions per interaction event
f = ROOT.TCanvas("f")
x_start = 0
x_end = 10
bins = 10

GEM_pion_QEL = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("GEM_pion_QEL", "Pion Amounts Per Interaction Type in GEM21 Model", bins, x_start, x_end), "num_pions")
GEM_pion_QEL.SetLineColor(ROOT.kViolet)
GEM_pion_QEL.Draw("same")

GEM_pion_MEC = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("GEM_pion_MEC", "", bins, x_start, x_end), "num_pions")
GEM_pion_MEC.SetLineColor(ROOT.kRed)
GEM_pion_MEC.Draw("same")

GEM_pion_RES = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("GEM_pion_RES", "", bins, x_start, x_end), "num_pions")
GEM_pion_RES.SetLineColor(ROOT.kGreen)
GEM_pion_RES.Draw("same")

GEM_pion_DIS = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("GEM_pion_DIS", "", bins, x_start, x_end), "num_pions")
GEM_pion_DIS.SetLineColor(ROOT.kBlue)
GEM_pion_DIS.Draw("same")

GEM_pion_none = df_gst_GEM21_11b_all.Filter("qel==0 && mec==0 && res==0 && dis==0").Histo1D(("GEM_pion_none", "", bins, x_start, x_end), "num_pions")
GEM_pion_none.SetLineColor(ROOT.kGray)
GEM_pion_none.Draw("same")

GEM_pion_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
GEM_pion_leg.AddEntry("GEM_pion_QEL", "QEL", "l")
GEM_pion_leg.AddEntry("GEM_pion_MEC", "MEC", "l")
GEM_pion_leg.AddEntry("GEM_pion_RES", "RES", "l")
GEM_pion_leg.AddEntry("GEM_pion_DIS", "DIS", "l")
GEM_pion_leg.AddEntry("GEM_pion_none", "Uncatagorized", "l")

GEM_pion_leg.Draw()
f.Draw()

g = ROOT.TCanvas("g")

G18_pion_QEL = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("G18_pion_QEL", "Pion Amounts Per Interaction Type in G18 Model", bins, x_start, x_end), "num_pions")
G18_pion_QEL.SetLineColor(ROOT.kViolet)
G18_pion_QEL.Draw("same")

G18_pion_MEC = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("G18_pion_MEC", "", bins, x_start, x_end), "num_pions")
G18_pion_MEC.SetLineColor(ROOT.kRed)
G18_pion_MEC.Draw("same")

G18_pion_RES = df_gst_G18_02a_all.Filter("res==1").Histo1D(("G18_pion_RES", "", bins, x_start, x_end), "num_pions")
G18_pion_RES.SetLineColor(ROOT.kGreen)
G18_pion_RES.Draw("same")

G18_pion_DIS = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("G18_pion_DIS", "", bins, x_start, x_end), "num_pions")
G18_pion_DIS.SetLineColor(ROOT.kBlue)
G18_pion_DIS.Draw("same")

G18_pion_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_pion_none", "", bins, x_start, x_end), "num_pions")
G18_pion_none.SetLineColor(ROOT.kGray)
G18_pion_none.Draw("same")

G18_pion_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
G18_pion_leg.AddEntry("G18_pion_QEL", "QEL", "l")
G18_pion_leg.AddEntry("G18_pion_MEC", "MEC", "l")
G18_pion_leg.AddEntry("G18_pion_RES", "RES", "l")
G18_pion_leg.AddEntry("G18_pion_DIS", "DIS", "l")
G18_pion_leg.AddEntry("G18_pion_none", "Uncatagorized", "l")

G18_pion_leg.Draw()
g.Draw()

Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used


In [5]:
#num of protons per interaction event
h = ROOT.TCanvas("h")
x_start = 0
x_end = 10
bins = 10

GEM_proton_QEL = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("GEM_proton_QEL", "Proton Amounts Per Interaction Type in GEM21 Model", bins, x_start, x_end), "nfp")
GEM_proton_QEL.SetLineColor(ROOT.kViolet)
GEM_proton_QEL.Draw("same")

GEM_proton_MEC = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("GEM_proton_MEC", "", bins, x_start, x_end), "nfp")
GEM_proton_MEC.SetLineColor(ROOT.kRed)
GEM_proton_MEC.Draw("same")

GEM_proton_RES = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("GEM_proton_RES", "", bins, x_start, x_end), "nfp")
GEM_proton_RES.SetLineColor(ROOT.kGreen)
GEM_proton_RES.Draw("same")

GEM_proton_DIS = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("GEM_proton_DIS", "", bins, x_start, x_end), "nfp")
GEM_proton_DIS.SetLineColor(ROOT.kBlue)
GEM_proton_DIS.Draw("same")

GEM_proton_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("GEM_proton_none", "", bins, x_start, x_end), "nfp")
GEM_proton_none.SetLineColor(ROOT.kGray)
GEM_proton_none.Draw("same")

GEM_proton_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
GEM_proton_leg.AddEntry("GEM_proton_QEL", "QEL", "l")
GEM_proton_leg.AddEntry("GEM_proton_MEC", "MEC", "l")
GEM_proton_leg.AddEntry("GEM_proton_RES", "RES", "l")
GEM_proton_leg.AddEntry("GEM_proton_DIS", "DIS", "l")
GEM_proton_leg.AddEntry("GEM_proton_none", "Uncatagorized", "l")

GEM_proton_leg.Draw()
h.Draw()

i = ROOT.TCanvas("i")

G18_proton_QEL = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("G18_proton_QEL", "Proton Amounts Per Interaction Type in G18 Model", bins, x_start, x_end), "nfp")
G18_proton_QEL.SetLineColor(ROOT.kViolet)
G18_proton_QEL.Draw("same")

G18_proton_MEC = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("G18_proton_MEC", "", bins, x_start, x_end), "nfp")
G18_proton_MEC.SetLineColor(ROOT.kRed)
G18_proton_MEC.Draw("same")

G18_proton_RES = df_gst_G18_02a_all.Filter("res==1").Histo1D(("G18_proton_RES", "", bins, x_start, x_end), "nfp")
G18_proton_RES.SetLineColor(ROOT.kGreen)
G18_proton_RES.Draw("same")

G18_proton_DIS = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("G18_proton_DIS", "", bins, x_start, x_end), "nfp")
G18_proton_DIS.SetLineColor(ROOT.kBlue)
G18_proton_DIS.Draw("same")

G18_proton_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_proton_none", "", bins, x_start, x_end), "nfp")
G18_proton_none.SetLineColor(ROOT.kGray)
G18_proton_none.Draw("same")

G18_proton_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
G18_proton_leg.AddEntry("G18_proton_QEL", "QEL", "l")
G18_proton_leg.AddEntry("G18_proton_MEC", "MEC", "l")
G18_proton_leg.AddEntry("G18_proton_RES", "RES", "l")
G18_proton_leg.AddEntry("G18_proton_DIS", "DIS", "l")
G18_proton_leg.AddEntry("G18_proton_none", "Uncatagorized", "l")

G18_proton_leg.Draw()
i.Draw()

Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used


In [6]:
#num of neutrons per interaction event
j = ROOT.TCanvas("j")
x_start = 0
x_end = 10
bins = 10

GEM_neutron_QEL = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("GEM_neutron_QEL", "Neutron Amounts Per Interaction Type in GEM21 Model", bins, x_start, x_end), "nfn")
GEM_neutron_QEL.SetLineColor(ROOT.kViolet)
GEM_neutron_QEL.Draw("same")

GEM_neutron_MEC = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("GEM_neutron_MEC", "", bins, x_start, x_end), "nfn")
GEM_neutron_MEC.SetLineColor(ROOT.kRed)
GEM_neutron_MEC.Draw("same")

GEM_neutron_RES = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("GEM_neutron_RES", "", bins, x_start, x_end), "nfn")
GEM_neutron_RES.SetLineColor(ROOT.kGreen)
GEM_neutron_RES.Draw("same")

GEM_neutron_DIS = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("GEM_neutron_DIS", "", bins, x_start, x_end), "nfn")
GEM_neutron_DIS.SetLineColor(ROOT.kBlue)
GEM_neutron_DIS.Draw("same")

GEM_neutron_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("GEM_neutron_none", "", bins, x_start, x_end), "nfn")
GEM_neutron_none.SetLineColor(ROOT.kGray)
GEM_neutron_none.Draw("same")

GEM_neutron_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
GEM_neutron_leg.AddEntry("GEM_neutron_QEL", "QEL", "l")
GEM_neutron_leg.AddEntry("GEM_neutron_MEC", "MEC", "l")
GEM_neutron_leg.AddEntry("GEM_neutron_RES", "RES", "l")
GEM_neutron_leg.AddEntry("GEM_neutron_DIS", "DIS", "l")
GEM_neutron_leg.AddEntry("GEM_neutron_none", "Uncatagorized", "l")

GEM_neutron_leg.Draw()
j.Draw()

k = ROOT.TCanvas("k")

G18_neutron_QEL = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("G18_neutron_QEL", "Neutron Amounts Per Interaction Type in G18 Model", bins, x_start, x_end), "nfn")
G18_neutron_QEL.SetLineColor(ROOT.kViolet)
G18_neutron_QEL.Draw("same")

G18_neutron_MEC = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("G18_neutron_MEC", "", bins, x_start, x_end), "nfn")
G18_neutron_MEC.SetLineColor(ROOT.kRed)
G18_neutron_MEC.Draw("same")

G18_neutron_RES = df_gst_G18_02a_all.Filter("res==1").Histo1D(("G18_neutron_RES", "", bins, x_start, x_end), "nfn")
G18_neutron_RES.SetLineColor(ROOT.kGreen)
G18_neutron_RES.Draw("same")

G18_neutron_DIS = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("G18_neutron_DIS", "", bins, x_start, x_end), "nfn")
G18_neutron_DIS.SetLineColor(ROOT.kBlue)
G18_neutron_DIS.Draw("same")

G18_neutron_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_neutron_none", "", bins, x_start, x_end), "nfn")
G18_neutron_none.SetLineColor(ROOT.kGray)
G18_neutron_none.Draw("same")

G18_neutron_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
G18_neutron_leg.AddEntry("G18_neutron_QEL", "QEL", "l")
G18_neutron_leg.AddEntry("G18_neutron_MEC", "MEC", "l")
G18_neutron_leg.AddEntry("G18_neutron_RES", "RES", "l")
G18_neutron_leg.AddEntry("G18_neutron_DIS", "DIS", "l")
G18_neutron_leg.AddEntry("G18_neutron_none", "Uncatagorized", "l")

G18_neutron_leg.Draw()
k.Draw()

Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used


In [12]:
#relativistic kinetic energy in diff interaction types without kef cut or particle selection
l = ROOT.TCanvas("l")
bins = 100
x_start = 0
x_end = 0.5
#df_gst_GEM21_11b_hadke = df_gst_GEM21_11b_all.Filter(f"kef>{hadron_accept_ke}")
gem_ke_p_qel = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("gem_ke_p_qel", "Kinetic Energy In GEM21 Model", bins, x_start, x_end), f"kef")
gem_ke_p_qel.SetLineColor(ROOT.kViolet)
gem_ke_p_qel.Draw("same")

gem_ke_p_mec = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("gem_ke_p_mec", "", bins, x_start, x_end), f"kef")
gem_ke_p_mec.SetLineColor(ROOT.kRed)
gem_ke_p_mec.Draw("same")

gem_ke_p_res = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("gem_ke_p_res", "", bins, x_start, x_end), f"kef")
gem_ke_p_res.SetLineColor(ROOT.kGreen)
gem_ke_p_res.Draw("same")

gem_ke_p_dis = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("gem_ke_p_dis", "", bins, x_start, x_end), f"kef")
gem_ke_p_dis.SetLineColor(ROOT.kBlue)
gem_ke_p_dis.Draw("same")

gem_ke_p_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("gem_ke_p_none", "", bins, x_start, x_end), f"kef")
gem_ke_p_none.SetLineColor(ROOT.kGray)
gem_ke_p_none.Draw("same")

gem_ke_p_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
gem_ke_p_leg.AddEntry("gem_ke_p_qel", "QEL", "l")
gem_ke_p_leg.AddEntry("gem_ke_p_mec", "MEC", "l")
gem_ke_p_leg.AddEntry("gem_ke_p_res", "RES", "l")
gem_ke_p_leg.AddEntry("gem_ke_p_dis", "DIS", "l")
gem_ke_p_leg.AddEntry("gem_ke_p_none", "Uncatagorized", "l")

gem_ke_p_leg.Draw()
l.SetLogy()
l.Draw()

m = ROOT.TCanvas("m")
g18_ke_p_qel = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("g18_ke_p_qel", "Kinetic Energy In G18 Model", bins, x_start, x_end) ,f"kef")
g18_ke_p_qel.SetLineColor(ROOT.kViolet)
g18_ke_p_qel.Draw("same")

g18_ke_p_mec = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("g18_ke_p_mec", "", bins, x_start, x_end) ,f"kef")
g18_ke_p_mec.SetLineColor(ROOT.kRed)
g18_ke_p_mec.Draw("same")

g18_ke_p_res = df_gst_G18_02a_all.Filter("res==1").Histo1D(("g18_ke_p_res", "", bins, x_start, x_end) ,f"kef")
g18_ke_p_res.SetLineColor(ROOT.kGreen)
g18_ke_p_res.Draw("same")

g18_ke_p_dis = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("g18_ke_p_dis", "", bins, x_start, x_end) ,f"kef")
g18_ke_p_dis.SetLineColor(ROOT.kBlue)
g18_ke_p_dis.Draw("same")

g18_ke_p_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("g18_ke_p_none", "", bins, x_start, x_end) ,f"kef")
g18_ke_p_none.SetLineColor(ROOT.kGray)
g18_ke_p_none.Draw("same")

g18_ke_p_leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)
g18_ke_p_leg.AddEntry("g18_ke_p_qel", "QEL", "l")
g18_ke_p_leg.AddEntry("g18_ke_p_mec", "MEC", "l")
g18_ke_p_leg.AddEntry("g18_ke_p_res", "RES", "l")
g18_ke_p_leg.AddEntry("g18_ke_p_dis", "DIS", "l")
g18_ke_p_leg.AddEntry("g18_ke_p_none", "Uncatagorized", "l")

g18_ke_p_leg.Draw()
m.SetLogy()
m.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: l
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
Warning in <TCanvas::Constructor>: Deleting canvas with same name: m
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used


In [ ]:
#everything from here below is trials that do not work :(
#problems: cannot use pdgf cut or ke cuts correctly, example from other notebook does not work? see below
#questions: difference between definition of kef_proton.size() and nfp?

In [ ]:
#filter doesnt work
a = ROOT.TCanvas("a")
bins = 100
x_start = 0
x_end = 10
gem_ke_p_qel = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("gem_ke_p_qel", "Kinetic Energy of Protons In GEM21 Model", bins, x_start, x_end), "kef_above_threshold_protons")
gem_ke_p_qel.SetLineColor(ROOT.kViolet)
gem_ke_p_qel.Draw("same")
a.Draw()

In [ ]:
#trying to plot kinetic energy of protons but cannot call protons specifically
l = ROOT.TCanvas("l")
bins = 100
x_start = 0
x_end = 0.5
df_gst_GEM21_11b_kep = df_gst_GEM21_11b_all.Filter(f"kef(pdgf==2112)")
bruh = df_gst_GEM21_11b_kep.Histo1D(("", "", bins, x_start, x_end), "ptl")
#gem_ke_p_qel = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("gem_ke_p_qel", "Kinetic Energy of idk In GEM21 Model", bins, x_start, x_end), f"kef")
#gem_ke_p_qel.SetLineColor(ROOT.kViolet)
#gem_ke_p_qel.Draw("same")


#example from wesley's notebook, doesnt work either?
#h_kef_p_all = df_gst_GEM21_11b_all.Histo1D(("", "", 100, 0, 1.0), "kef_proton")
#h_kef_p_all.Draw()

h_kef_p_max = df_gst_GEM21_11b_all.Define("kef_proton_max", "Max(kef(pdgf==2212)").Histo1D(("", "", bins, x_start, x_end), "kef_proton_max")
h_kef_p_max.Draw()
bruh.Draw()
l.Draw()
#df_gst_mine = df_gst_mine.Define("kef_above_threshold_protons",f"kef[kef>{hadron_accept_ke}&&pdgf==2212]")

In [ ]:
#plot with cuts on lep pt and angle by interaction type and ptl, ptl should be pdgf code?  
a = ROOT.TCanvas("a")
x_start = 0
x_end = 1.5
bins = 100
df_gst_GEM21_11b_accept = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")
h_lep_pt_QEL = df_gst_GEM21_11b_accept.Filter("qel==1").Histo1D(("h_lep_pt_QEL", "Counts in Acceptance Range in GEM Model Per Interaction Type", bins, x_start, x_end), "ptl")
h_lep_pt_QEL.SetLineColor(ROOT.kBlue)
h_lep_pt_QEL.Draw("same")

h_lep_pt_MEC = df_gst_GEM21_11b_accept.Filter("mec==1").Histo1D(("h_lep_pt_MEC", "", bins, x_start, x_end), "ptl")
h_lep_pt_MEC.SetLineColor(ROOT.kRed)
h_lep_pt_MEC.Draw("same")

h_lep_pt_RES = df_gst_GEM21_11b_accept.Filter("res==1").Histo1D(("h_lep_pt_RES", "", bins, x_start, x_end), "ptl")
h_lep_pt_RES.SetLineColor(ROOT.kGreen)
h_lep_pt_RES.Draw("same")

h_lep_pt_DIS = df_gst_GEM21_11b_accept.Filter("dis==1").Histo1D(("h_lep_pt_DIS", "", bins, x_start, x_end), "ptl")
h_lep_pt_DIS.SetLineColor(ROOT.kViolet)
h_lep_pt_DIS.Draw("same")

h_lep_pt_none = df_gst_GEM21_11b_accept.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("h_lep_pt_none", "", bins, x_start, x_end), "ptl")
h_lep_pt_none.SetLineColor(ROOT.kGray)
h_lep_pt_none.Draw("same")

leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)

leg.AddEntry("h_lep_pt_QEL", "QE", "l")
leg.AddEntry("h_lep_pt_MEC", "MEC", "l")
leg.AddEntry("h_lep_pt_RES", "RES", "l")
leg.AddEntry("h_lep_pt_DIS", "DIS", "l")
leg.AddEntry("h_lep_pt_none", "Uncatagorized", "l")
leg.Draw()
a.Draw()

d = ROOT.TCanvas("d")
df_gst_G18_02a_accept = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")
h18_lep_pt_QEL = df_gst_G18_02a_accept.Filter("qel==1").Histo1D(("h_lep_pt_QEL", "Counts in Acceptance Range in G18 Model Per Interaction Type", bins, x_start, x_end), "ptl")
h18_lep_pt_QEL.SetLineColor(ROOT.kBlue)
h18_lep_pt_QEL.Draw("same")

h18_lep_pt_MEC = df_gst_G18_02a_accept.Filter("mec==1").Histo1D(("h_lep_pt_MEC", "", bins, x_start, x_end), "ptl")
h18_lep_pt_MEC.SetLineColor(ROOT.kRed)
h18_lep_pt_MEC.Draw("same")

h18_lep_pt_RES = df_gst_G18_02a_accept.Filter("res==1").Histo1D(("h_lep_pt_RES", "", bins, x_start, x_end), "ptl")
h18_lep_pt_RES.SetLineColor(ROOT.kGreen)
h18_lep_pt_RES.Draw("same")

h18_lep_pt_DIS = df_gst_G18_02a_accept.Filter("dis==1").Histo1D(("h_lep_pt_DIS", "", bins, x_start, x_end), "ptl")
h18_lep_pt_DIS.SetLineColor(ROOT.kViolet)
h18_lep_pt_DIS.Draw("same")

h18_lep_pt_none = df_gst_G18_02a_accept.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("h18_lep_pt_none", "", bins, x_start, x_end), "ptl")
h18_lep_pt_none.SetLineColor(ROOT.kGray)
h18_lep_pt_none.Draw("same")

legg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)

legg.AddEntry("h_lep_pt_QEL", "QE", "l")
legg.AddEntry("h_lep_pt_MEC", "MEC", "l")
legg.AddEntry("h_lep_pt_RES", "RES", "l")
legg.AddEntry("h_lep_pt_DIS", "DIS", "l")
legg.AddEntry("h18_lep_pt_none", "Uncatagorized", "l")
legg.Draw()
d.Draw()

In [ ]:
df_bruh = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")
df_bruh = df_bruh.Define("kef_proton","kef[pdgf==2212]").Define("n_proton","kef_proton.size()")
histo_bruh = df_bruh.Histo2D(("","",100,0,2.0,10,0,10), "kef_proton","n_proton")
histo_bruh.Draw("colz")
d.Draw()

In [ ]:
#acceptange range without pions, ptl?
d = ROOT.TCanvas("d")
df_gst_G18_02a_accept = df_gst_G18_02a_all.Filter(f"ptl>{0} && abs(thetazl)<{lep_accept_angle}")
df_gst_G18_02a_nopions = df_gst_G18_02a_accept.Filter("num_pions==0")
h18_lep_pt_QELp = df_gst_G18_02a_nopions.Filter("qel==1").Histo1D(("h_lep_pt_QELp", "Angles in Acceptance Range", 100, 0, 3.0), "ptl")
h18_lep_pt_QELp.SetLineColor(ROOT.kBlue)
h18_lep_pt_QELp.Draw("same")

h18_lep_pt_MECp = df_gst_G18_02a_nopions.Filter("mec==1").Histo1D(("h_lep_pt_MECp", "", 100, 0, 3.0), "ptl")
h18_lep_pt_MECp.SetLineColor(ROOT.kRed)
h18_lep_pt_MECp.Draw("same")

h18_lep_pt_RESp = df_gst_G18_02a_nopions.Filter("res==1").Histo1D(("h_lep_pt_RESp", "", 100, 0, 3.0), "ptl")
h18_lep_pt_RESp.SetLineColor(ROOT.kGreen)
h18_lep_pt_RESp.Draw("same")

h18_lep_pt_DISp = df_gst_G18_02a_nopions.Filter("dis==1").Histo1D(("h_lep_pt_DISp", "", 100, 0, 3.0), "ptl")
h18_lep_pt_DISp.SetLineColor(ROOT.kViolet)
h18_lep_pt_DISp.Draw("same")

legp = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)

legp.AddEntry("h_lep_pt_QELp", "QE", "l")
legp.AddEntry("h_lep_pt_MECp", "MEC", "l")
legp.AddEntry("h_lep_pt_RESp", "RES", "l")
legp.AddEntry("h_lep_pt_DISp", "DIS", "l")
legp.Draw()
d.Draw()
print(h18_lep_pt_QELp.Integral(), h18_lep_pt_MECp.Integral(), h18_lep_pt_RESp.Integral(), h18_lep_pt_DISp.Integral())
#num of events, use to find fractions, plot as a function of cuts, 0 cut to 1GeV or 0.5GeV cut,

In [ ]:
#data for cuts, how to range cuts from 0 to 1 gev?
bins = 100
x_start = 0
x_end = 3.0
df_gst_G18_02a_zero = df_gst_G18_02a_all.Filter("num_pions==0").Filter(f"ptl>{0} && abs(thetazl)<{lep_accept_angle}")
h18_lep_pt_QELz = df_gst_G18_02a_zero.Filter("qel==1").Histo1D(("h_lep_pt_QELz", "", bins, x_start, x_end), "ptl")
h18_lep_pt_MECz = df_gst_G18_02a_zero.Filter("mec==1").Histo1D(("h_lep_pt_MECz", "", bins, x_start, x_end), "ptl")
h18_lep_pt_RESz = df_gst_G18_02a_zero.Filter("res==1").Histo1D(("h_lep_pt_RESz", "", bins, x_start, x_end), "ptl")
h18_lep_pt_DISz = df_gst_G18_02a_zero.Filter("dis==1").Histo1D(("h_lep_pt_DISz", "", bins, x_start, x_end), "ptl")
zero_gev = [h18_lep_pt_QELz.Integral(), h18_lep_pt_MECz.Integral(), h18_lep_pt_RESz.Integral(), h18_lep_pt_DISz.Integral()]
total = zero_gev[0]+zero_gev[1]+zero_gev[2]+zero_gev[3]
print(zero_gev, "total events", total)
print("QEL Fraction", zero_gev[0]/total, "MEC Fraction", zero_gev[1]/total)
print("RES Fraction",zero_gev[2]/total, "DIS Fraction", zero_gev[3]/total)

df_gst_G18_02a_one = df_gst_G18_02a_all.Filter("num_pions==0").Filter(f"ptl>{1} && abs(thetazl)<{lep_accept_angle}")
h18_lep_pt_QELo = df_gst_G18_02a_one.Filter("qel==1").Histo1D(("h_lep_pt_QELo", "", bins, x_start, x_end), "ptl")
h18_lep_pt_MECo = df_gst_G18_02a_one.Filter("mec==1").Histo1D(("h_lep_pt_MECo", "", bins, x_start, x_end), "ptl")
h18_lep_pt_RESo = df_gst_G18_02a_one.Filter("res==1").Histo1D(("h_lep_pt_RESo", "", bins, x_start, x_end), "ptl")
h18_lep_pt_DISo = df_gst_G18_02a_one.Filter("dis==1").Histo1D(("h_lep_pt_DISo", "", bins, x_start, x_end), "ptl")
one_gev = [h18_lep_pt_QELo.Integral(), h18_lep_pt_MECo.Integral(), h18_lep_pt_RESo.Integral(), h18_lep_pt_DISo.Integral()]
total_one = one_gev[0]+one_gev[1]+one_gev[2]+one_gev[3]
print(one_gev, "total events", total_one)
print("QEL Fraction", one_gev[0]/total_one, "MEC Fraction", one_gev[1]/total_one)
print("RES Fraction", one_gev[2]/total_one, "DIS Fraction", one_gev[3]/total_one)

In [ ]:
#move old graphs into this format
#plot of fraction of events per interaction type on cut of pt